In [2]:
import gurobipy as gp
import itertools as it
import math

"""
implementation of solving profit maximization problem
author(s): 
yk796@cornell.edu
nd396@cornell.edu
"""

'\nimplementation of solving profit maximization problem\nauthor(s): \nyk796@cornell.edu\nnd396@cornell.edu\n'

In [50]:
# input
n_nodes = 4
n_alternative = 2


p_sen = 1
 
bpr_func = {1: {(1, 2): lambda x: 20, (1, 3): lambda x: x/10 , (2, 4): lambda x: x/10, (3, 4): lambda x: 20}, 
            2: {(1, 2): lambda x: 20, (1, 3): lambda x: 20 , (2, 4): lambda x: 20, (3, 4): lambda x: 20}}

nodes = list(range(1, n_nodes+1))
alternatives = list(range(1, n_alternative+1))
arcs = [(1,2), (2,4), (1,3), (3,4)] #list(it.permutations(nodes, 2))
#ods = list(it.permutations(nodes, 2))
# ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]
ods = [(1,4)]
demand = {(1, 4): 200}
T = {key:10 for key in list(it.product(ods, alternatives))}
ASC = {key:0 for key in list(it.product(ods, alternatives))}


def create_route(od):
    (o, d) = od
    # find all possible routes
    return (o, d)
    

def indicator(arc, route): 
    '''
    To check if an arc is in route
    '''
    # Check if arc is a tuple and has 2 elements
    if not isinstance(arc, tuple) or len(arc) != 2:
        raise ValueError("Arc must be a tuple with 2 elements")

    # Iterate through the route and check each pair
    for i in range(len(route) - 1):
        if route[i] == arc[0] and route[i+1] == arc[1]:
            return True
    return False




In [51]:
def profit_maximization(n_nodes, arcs, n_alternative, ods, demand, T, ASC, bpr_func):
    eps = 1e-3

    m = gp.Model()
    m.Params.NonConvex = 2 
    m.Params.DualReductions = 0 # to determine if the model is infeasible or unbounded
    # m.Params.OutputFlag = 0
    # m.Params.NonConvex = 2 # because of profit_extracting_log term is concave, we need to tell gurobi
    # that this is not a concave model, although it is. 
    #m.Params.MIPGap = 0


    m._T = T
    m._ASC = ASC
    m._bpr_func = bpr_func
    m._nodes = list(range(1, n_nodes+1))
    m._alternatives = list(range(1, n_alternative+1))
    m._arcs = arcs 
    m._ods = ods #[(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]
    m._routes = {(1,4): [(1,2,4), (1,3,4)]} # {key:create_route(key) for key in m._ods}
    

    m._theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=eps, ub=1, name='theta')
    m._y_vars = m.addVars(list(it.product(m._arcs, m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='y') # in fully connected graph, y=z
    
    # for yvar in m._y_vars.values():
    #     yvar.start = 100

    for (o,d) in m._ods:
        m._z_vars = m.addVars(list(it.product(m._routes[(o,d)], m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='z') # in fully connected graph, y=z

    
    # create auxiliary variables to deal with non-linear objective function

    m._fvars = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, lb=0, name='f') 

    # # Dictionary for initial values
    # initial_values = {(1, 2): 100, (2, 4): 100, (1, 3):100, (3, 4): 100}

    # # Loop to set initial values
    # for arc, init_val in initial_values.items():
    #     m._fvars[arc].Start = init_val
    
    #m._ln_theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='ln_theta')
    m._theta_lntheta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='theta_ln_theta') #define theta * ln(theta)
    m._congest_tt = m.addVars(list(it.product(m._ods, m._alternatives)), lb = 0, vtype=gp.GRB.CONTINUOUS, name='congested_travel_time') 


    m._ind = m.addVars(m._ods, vtype=gp.GRB.BINARY, name="ind")
    m._profit_extracting = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb= 0, ub=1, name='extracting')
    m._profit_extracting_log = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb = -float('inf'), ub=0, name='extracting_log')
    m._profit_extracting_term = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='extracting_term')
    m._F = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, name='F')
    m._G = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, name='G')


    """add constraints"""
    # relationship between theta and z
    for j in m._alternatives:
        for (s, t) in m._ods:
            lhs = gp.quicksum(m._z_vars[r, j] for r in m._routes[(s, t)])
            rhs = m._theta_vars[(s, t), j]
            m.addConstr(lhs == rhs, name = "constraint Q (a)")

    for (s, t) in m._ods:
        lhs = gp.quicksum(m._theta_vars[(s, t), j] for j in m._alternatives)
        rhs = 1 #- eps
        m.addConstr(lhs == rhs, name = "constraint Q (b)") # it should be <= for rebalancing. Now, we will ignore it

    for j in m._alternatives:
        for a in m._arcs:
            for (s, t) in m._ods:
                m.addConstr(gp.quicksum([demand[s,t] * m._z_vars[r, j] for r in m._routes[(s, t)] if indicator(a, r)]) == m._y_vars[a, (s, t), j], name = "constraint Q (c)") 


    # relationship to f_a
    for j in alternatives:
        for a in m._arcs:
            lhs = m._fvars[j,a]
            rhs = gp.quicksum(m._y_vars[a, (s, t), j] for (s, t) in m._ods)
            m.addConstr(lhs == rhs, name = "equation Q (d)")

    # flow conservation
    for j in m._alternatives:
        for v in m._nodes:
            if (v != s) and (v != t):
                lhs = gp.quicksum(m._y_vars[(a, b) , (s, t), j] for (a,b) in m._arcs if v == a)
                rhs = gp.quicksum(m._y_vars[(a, b) , (s, t), j] for (a,b) in m._arcs if v == b)
                m.addConstr(lhs == rhs, name = "constraint Q (e)")


    bins = 100
    xs = [1/bins*i for i in range(bins+1)]
    ys = [p*math.log(p) if p != 0 else 0 for p in xs]
    # objective function
    for j in m._alternatives:
        for (s, t) in m._ods:
            #m.addConstr(m._theta_vars[(s, t), j] * m._inv_theta_vars[(s, t), j] == 1, name = "inv_theta") 
            #m.addGenConstrLog(m._theta_vars[(s, t), j], m._ln_theta_vars[(s, t), j], name = "ln_theta") 
            m.addGenConstrPWL(m._theta_vars[(s, t), j], m._theta_lntheta_vars[(s, t), j], xs, ys, "pwl")

    # for (s, t) in m._ods:
    #     m.addConstr(m._profit_extracting[s,t] == 1 - gp.quicksum(m._theta_vars[(s, t), j] for j in m._alternatives), name ='extract')
    #     m.addGenConstrLog(m._profit_extracting[s,t], m._profit_extracting_log[s,t], name = "ln_profit")
    #     for j in m._alternatives:
    #         m.addConstr(m._profit_extracting_term[(s, t), j] == m._theta_vars[(s, t), j] * m._profit_extracting[s,t])
            
    for j in m._alternatives:
        for (s, t) in m._ods:
            #m.addConstr(m._congest_tt[(s, t), j] == gp.quicksum(gp.quicksum(m._z_vars[r, j]*m._F[a] for r in m._R[(s, t)] if indicator(a, r)) for a in m._arcs))

            m.addConstr(m._congest_tt[(s, t), j] == gp.quicksum(gp.quicksum(m._z_vars[r, j]*m._F[j,a] for a in m._arcs if indicator(a, r)) for r in m._routes[(s, t)]))


    # # TODO: include general bpr function instead of the current linear function
    # m.addConstr(m._F[(2,4)] == m._fvars[(2,4)]/10, name = "F_function")
    # m.addConstr(m._F[(1,3)] == m._fvars[(1,3)]/10, name = "F_function")
    # m.addConstr(m._F[(1,2)] == 20, name = "F_function")
    # m.addConstr(m._F[(3,4)] == 20, name = "F_function")

    # Define F and G
    for j in alternatives:
        for a in m._arcs:
            lhs = m._F[j,a]
            rhs = m._bpr_func[j][a](m._fvars[j,a])
            m.addConstr(lhs == rhs, name = "F_function")

            lhs = m._G[j,a]
            rhs = m._bpr_func[j][a](m._fvars[j,a])
            m.addConstr(lhs == rhs, name = "G_function")
            

    # # define objective function
    # m.setObjective(gp.quicksum(D[s,t]* gp.quicksum(m._theta_vars[(s, t), j] * (m._T[(s, t), j] - m._ASC[(s, t), j]) for j in m._alternatives) for (s, t) in m._ods) # objective function (a)
    #                )
    
    obj_util = gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._theta_vars[(s, t), j] * (m._T[(s, t), j] - m._ASC[(s, t), j]) for j in m._alternatives) for (s, t) in m._ods) # objective function (A)
    obj_congest = gp.quicksum(demand[s,t]/p_sen * m._congest_tt[(s, t), j] for j in m._alternatives for (s, t) in m._ods)
    obj_b = gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._theta_lntheta_vars[(s, t), j] for j in m._alternatives) for (s, t) in m._ods) # objective function (B)
    obj_c = - gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._profit_extracting_term[(s, t), j] for j in m._alternatives) for (s, t) in m._ods) # objective function (C) 
    obj_d = gp.quicksum(gp.quicksum(m._G[j,a]* m._y_vars[a , (s, t), j] for a in m._arcs) for j in m._alternatives for (s, t) in m._ods) # objective function (D)
    # define objective function
    m.setObjective(obj_util + obj_congest + obj_b + obj_c) # +  + obj_d

    m.update()
    m.optimize()

    # m.computeIIS() # this helps us to identify constraints that are responsible to make the model infeasible.
    # m.write("model.ilp")

    for v in m.getVars():
        print(f"{v.VarName} = {v.X}")


    if m.Status == 3:
        return None, None
    else:
        return m


result = profit_maximization(n_nodes, arcs, n_alternative, ods, demand, T, ASC, bpr_func)



Set parameter NonConvex to value 2
Set parameter DualReductions to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6244 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 39 rows, 47 columns and 68 nonzeros
Model fingerprint: 0x8dfdb364
Model has 2 quadratic constraints
Model has 2 general constraints
Variable types: 46 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+02, 2e+03]
  Bounds range     [1e-03, 1e+00]
  RHS range        [1e+00, 2e+01]
  PWLCon x range   [1e-02, 1e+00]
  PWLCon y range   [1e-02, 4e-01]
Presolve added 0 rows and 183 columns
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 46 rows, 232 columns, 2971 nonzeros
Presolved model has 2 bilinear constraint(s)
Va

In [52]:
# open a file in write mode
with open("output.txt", "w") as file:
    for v in result.getVars():
        # Write each line to the file
        file.write(f"{v.VarName} = {v.X}\n")

# calculating profit
